<a href="https://colab.research.google.com/github/sinaans/GeoSpatial/blob/main/Foursquare_Point(All).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Function to get Coordinate of a place based on the given information
# Importing libraries

import subprocess

try:
    import geopy
except ImportError:
    print('geopy package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geopy'])

from geopy.geocoders import Nominatim # Module to convert an address into latitude and longitude variable


def findLoc(placeName):
  geolocator = Nominatim(user_agent='foursquare_agent')
  location = geolocator.geocode(placeName)
  print(location.address)
  print('Latitude:', location.latitude, ', Longitude:', location.longitude)
  coordinateWGS = [location.latitude, location.longitude]
  return coordinateWGS

In [ ]:
# Getting coordinate of the Eeasmus Bridge in Rotterdam based on English name 
wgs = findLoc('Erasmus Bridge')

Erasmusbrug, Prins Hendrikstraat, Noordereiland, Feijenoord, Rotterdam, Zuid-Holland, Nederland, 3071 LH, Nederland
Latitude: 51.90961245 , Longitude: 4.485461234217793


In [ ]:
# Getting coordinate of the Eeasmus Bridge in Rotterdam based on Dutch name 
wgs1 = findLoc('Erasmusbrug')

Erasmusbrug, Prins Hendrikstraat, Noordereiland, Feijenoord, Rotterdam, Zuid-Holland, Nederland, 3071 LH, Nederland
Latitude: 51.90961245 , Longitude: 4.485461234217793


In [ ]:
# Function to obtain some information of a all vendors around a particular location
# It is necessary to enter the FourSquare's API Key, Lat and Long (Decimal Degree), 
# Radius from the given point (m), and number of Records (for now it is Maximum 50).

import requests # Library to handle requestes
from pandas import json_normalize # Transforming json file into a pandas dataframe library


def searchPointAll(apiKey, latitude, longitude, radius, limit):

  # Creating a URL for FS API
  url = 'https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&limit={}'.format(latitude, longitude, radius, limit)

  if limit > 50:
    print("Maximum number of records cannot be more than 50.")
  else:
    headers={
        'Accept': 'application/json',
        'Authorization': apiKey
    }
    
    # Fetching data from the URL
    results = requests.get(url, headers=headers).json()

    initial_df = json_normalize(results['results'])
    filtered_columns = ['name', 'categories', 'geocodes.main.latitude', 'geocodes.main.longitude'] + [col for col in initial_df.columns if col.startswith('location.')] + ['fsq_id']
    filtered_df = initial_df.loc[:, filtered_columns]

    # Clean column names by keeping only last term
    filtered_df.columns = [column.split('.')[-1] for column in filtered_df.columns]

    # Remove rows that has no Latitude or longitude value
    filtered_df.dropna(subset=['latitude', 'longitude'], inplace=True)

    def get_ctgry_typ(row):
      try:
        categories_list = row['categories']
      except:
        categories_list = row['results.categories']
      
      if len(categories_list) == 0:
        return None
      else:
        return categories_list[0]['name']

    # Filter the category for each row
    filtered_df['categories'] = filtered_df.apply(get_ctgry_typ, axis=1)

    return filtered_df
    

In [ ]:
# Giving an Example for vendors around the Eiffel Tower
location = findLoc('Eiffel Tower')
radius = 500
limit = 50
apiKey = '' # Get this after generating API Key in your project in FQ

Tour Eiffel, 5, Avenue Anatole France, Quartier du Gros-Caillou, Paris 7e Arrondissement, Paris, Île-de-France, France métropolitaine, 75007, France
Latitude: 48.858260200000004 , Longitude: 2.2944990543196795


In [ ]:
# Eample for the Given Location
my_All_results = searchPointAll(apiKey, location[0], location[1], radius, limit)
my_All_results

,name,categories,latitude,longitude,address,admin_region,country,cross_street,formatted_address,locality,postcode,region,fsq_id
0,Le Jules Verne,Brasserie,48.858237,2.294416,5 avenue Gustave Eiffel,Île-de-France,FR,"Tour Eiffel, 2e étage","5 avenue Gustave Eiffel (Tour Eiffel, 2e étage...",Paris,75007,Île-de-France,4b5972b9f964a5208b8828e3
1,Bar à Champagne,Bar,48.858300,2.294626,5 avenue Anatole France,Île-de-France,FR,Sommet de la Tour Eiffel,5 avenue Anatole France (Sommet de la Tour Eif...,Paris,75007,Île-de-France,4d55537db4deb1f71d41037a
2,Restaurant 58 Tour Eiffel,French Restaurant,48.858328,2.294339,5 avenue Anatole France,Île-de-France,FR,"Tour Eiffel, 1er étage","5 avenue Anatole France (Tour Eiffel, 1er étag...",Paris,75007,Île-de-France,4e078807e4cdefcff6dce4f6
3,Buffet Tour Eiffel,Buffet,48.858257,2.294502,5 avenue Anatole France,Île-de-France,FR,"Tour Eiffel, 2e étage","5 avenue Anatole France (Tour Eiffel, 2e étage...",Paris,75007,Île-de-France,596289d759c4234a4ced3319
4,Ladurée,Dessert Shop,48.858381,2.294794,5 avenue Anatole France,Île-de-France,FR,,"5 avenue Anatole France, 75007 Paris",Paris,75007,Île-de-France,5de805768fc06100089a79e2
5,Boutique officielle,Gift Store,48.858357,2.294471,5 avenue Anatole France,Île-de-France,FR,,"5 avenue Anatole France, 75007 Paris",Paris,75007,Île-de-France,575886a5498eee35ef771349
6,Restaurant Jules Verne on top of the Eiffel Tower,Dining and Drinking,48.858401,2.294460,5 avenue Anatole France,Île-de-France,FR,"Tour Eiffel, 2e étage","5 avenue Anatole France (Tour Eiffel, 2e étage...",Paris,75007,Île-de-France,5c8239b2f870fd002cd36a17
7,Studio Gustave Eiffel,Movie Theater,48.858188,2.294607,5 avenue Anatole France,Île-de-France,FR,"Tour Eiffel, 1er étage","5 avenue Anatole France (Tour Eiffel, 1er étag...",Paris,75007,Île-de-France,5cac6d2782a750002c291a1b
8,Bar à Macarons,Pastry Shop,48.858104,2.294616,5 avenue Anatole France,Île-de-France,FR,,"5 avenue Anatole France, 75007 Paris",Paris,75007,Île-de-France,5d14bb96a18cd5002fc58027
9,La Bulle Parisienne,Restaurant,48.855899,2.298088,Champ de Mars 5 avenue Anatole France,Île-de-France,FR,,"Champ de Mars 5 avenue Anatole France, 75007 P...",Paris,75007,Île-de-France,5dc7f5aa7ebb9e00089f3d20
